In [48]:
# %load_ext autoreload
# %reload_ext autoreload
# %autoreload 2

In [49]:
import os
os.chdir('C:\\Users\\WDG1DCR\\Desktop\\my_projects\\data_analysis\\data_cleanse')

import pandas as pd
from prep import filter
from prep import sort
from plot import bar
from utility import generate

In [50]:
fileName = 'filelist_7_15'
df_og = pd.read_csv('datasets/' + fileName + '.csv', sep=';')

In [51]:
df = filter.sld(df_og)

In [52]:
dups = df[df.duplicated(['name_low'], keep=False)]
dups = dups.sort_values(['name_low'])

In [53]:
#only works with input '3' so far
paths = sort.generate_paths(dups, 3)

In [54]:
df_input_0 = paths.copy()

In [55]:
output_collection0 = {}
output_collection1 = {}
output_collection2 = {}

dropped_col_list0 = []
dropped_col_list1 = []
dropped_col_list2 = []

In [56]:
# this function is also intended to eventually be path qty independent
# aka wil work for x2 paths, x3 paths, x4 paths, etc
# right now it's set up for x3 paths only

def generate(df_target, dropped_col):
       
    output_collection1, dropped_col_list1 = sort.chunk2(df_target)

    temp_inc0 = 0

    rebuild_collection2 = {}
    for key in output_collection1.keys():
        output_collection2, dropped_col_list2 = sort.chunk2(output_collection1[key])

        df_rebuild2 = sort.rebuild(output_collection2, dropped_col_list2, 'path3')
        rebuild_collection2[key] = df_rebuild2

    #     temp_inc0 += 1
    #     if temp_inc0 >= 2:
    #         break

    df_rebuild1 = sort.rebuild(rebuild_collection2, dropped_col_list1, 'path2')
    df_rebuild1.insert(1, 'path1', dropped_col)
    columns_titles = ['name', 'path1', 'path2', 'path3']
    df_out=df_rebuild1.reindex(columns=columns_titles)

    sheetName = dropped_col.split('\\', -1)
    sheetName = sheetName[len(sheetName)-1]
    sheetName = sheetName[:25]

    return df_out, sheetName


In [57]:
output_collection0, dropped_col_list0 = sort.chunk2(df_input_0)

In [58]:
# choose how many directories to analyze
num = 20

df_out_collection = {}
sheetName_list = []

for i in range(num):
    
    dict_key_list = list(output_collection0.keys())
    dict_key = dict_key_list[i]
    
    df_target = output_collection0[dict_key].copy()
    dropped_col = dropped_col_list0[i]

    df_out, sheetName = generate(df_target, dropped_col)
    
    df_out_collection[i] = df_out
    sheetName_list.append(sheetName)

In [59]:
fname = "output_files\\output.xlsx"

df_out.to_excel(fname,
             sheet_name=sheetName)

with pd.ExcelWriter(fname) as writer: 
    for i in range(num):
        df_out_collection[i].to_excel(writer, sheet_name=sheetName_list[i])